# Tugas IF4070 - Representasi Pengetahuan dan Penalaran
# Implementasi Ripple Down Rules

# Simple RDR
**Simple RDR** is a ripple down rules implemented in Python.

## Setup
Assuming you've installed the latest version of Python (if not, guides for it are widely available),
1. ensure pip is installed by running `python -m ensurepip --upgrade`;
2. install the Python dependencies by running `pip install -r requirements.txt`.

In [ ]:
import random
import re

In [1]:
class Node:
    def __init__(self, precedent: set[str], antecedent: str, cornerstone: set[str], except_: "Node" = None, else_: "Node" = None, is_root: bool = False) -> None:
        self._precedent = precedent
        self._antecedent = antecedent
        self._cornerstone = cornerstone
        self._except = except_
        self._else = else_
        self._is_root = is_root

    def get_except(self) -> "Node":
        return self._except

    def get_else(self) -> "Node":
        return self._else
    
    def set_except(self, except_: "Node") -> None:
        self._except = except_
    
    def set_else(self, else_: "Node") -> None:
        self._else = else_
    
    def match_precedent(self, case: str) -> tuple[bool, str]:
        if self._is_root:
            return True, self._antecedent
        else:
            for statement in self._precedent:
                if statement.startswith("~"):
                    exec(f"{statement[1:]} = False", __locals=locals())
                else:
                    exec(f"{statement} = True", __locals=locals())
            
            try:
                eval(case)
                return True, self._antecedent
            except NameError:
                return False, self._antecedent
            finally:
                for statement in self._precedent:
                    if statement.startswith("~"):
                        exec(f"del {statement[1:]}", __locals=locals())
                    else:
                        exec(f"del {statement}", __locals=locals())

In [2]:
class Tree:
    def __init__(self, root: "Node"):
        self._root = root
    
    @staticmethod
    def _isolate_statement(case: str) -> set[str]:
        return set(re.findall(r"~?[A-z0-9]+\b", case))
    
    def _traverse_tree(self, case: str) -> None:
        current_node = self._root
        last_true = self._root
        
        while not current_node.get_except() and not current_node.get_else():
            if current_node.match_precedent(case)[0]:
                last_true = current_node
                current_node = current_node.get_except()
            else:
                current_node = current_node.get_else()
        
        antecedent = last_true[1]
        print(f"The system concludes that your case can be associated with the following: {antecedent}.")
        print(f"Do you agree? (y/n)")
        
        while True:
            agreement = input()
            if agreement.lower() in ("y", "n"):
                break
            else:
                print(f"Please input a valid option!")
        
        if agreement.lower() == "y":
            pass
        else:
            pass